# 4. Clustering With K-Means

## 설명
- 예측을 위한 feature engineering의 관점에서 unsupervised algorithm을 `feature discovery` 기법으로 생각할 수 있다.
- cluster labels를 feature로 추가함으로써 ML 모델이 복잡한 공간/근접 관계를 푸는 데 도움을 준다.
- cluster feature는 `categorical` feature다.

## 예시
<img src="assets/clustering.png" width="50%"/>

- 비선형 관계를 선형 모델이 학습하기는 너무 어렵다. 그런데 cluster별로 관계를 나누어 각 부분이 선형 관계를 이루도록 만듦으로써 모델이 쉽게 학습 할 수 있다.

## K-Means Clustering
- 유클리드 거리를 사용한다.
- scale에 매우 민감하기 때문에 어떻게 rescale할지 고민해야 한다. 몸무게와 키처럼 comparable하지 않은 features인 경우 rescale을 해주어야 한다.
- `centroids`라고 불리는 points를 feature-space에 위치시켜 cluster를 생성한다.
- 데이터셋의 각 point는 가장 가까운 거리에 위치한 centroid의 cluster에 할당된다.
- 작동
    1. assign points to the nearest cluster centroid
    2. move each centroid to minimize the distance to its points
    3. 1-2를 반복한다.
- parameters
    1. n_clusters: centroids의 개수
    2. max_iter: 위에 기재된 반복을 몇 번 수행할 것인지
    3. n_init: 몇 번 알고리즘을 수행할 것인지 -> 데이터셋의 각 points와 centroids 사이 거리 합이 최소인 clustering 결과를 반환한다.
- 클러스터가 많을 경우 max_iter를 크게, 데이터셋이 복잡할 경우 n_init을 크게 해야 한다.
- cluster labels가 의미 있는지 어떻게 아는가?
    - target이 cluster별로 분리되어 있어야 한다.
    
    <img src="assets/clustering2.png" width="50%"/>

## 실습

In [4]:
import pandas as pd
from sklearn.cluster import KMeans

In [19]:
df = pd.read_csv('./data/ames.csv')

In [7]:
df.columns

Index(['Order', 'PID', 'MS.SubClass', 'MS.Zoning', 'Lot.Frontage', 'Lot.Area',
       'Street', 'Alley', 'Lot.Shape', 'Land.Contour', 'Utilities',
       'Lot.Config', 'Land.Slope', 'Neighborhood', 'Condition.1',
       'Condition.2', 'Bldg.Type', 'House.Style', 'Overall.Qual',
       'Overall.Cond', 'Year.Built', 'Year.Remod.Add', 'Roof.Style',
       'Roof.Matl', 'Exterior.1st', 'Exterior.2nd', 'Mas.Vnr.Type',
       'Mas.Vnr.Area', 'Exter.Qual', 'Exter.Cond', 'Foundation', 'Bsmt.Qual',
       'Bsmt.Cond', 'Bsmt.Exposure', 'BsmtFin.Type.1', 'BsmtFin.SF.1',
       'BsmtFin.Type.2', 'BsmtFin.SF.2', 'Bsmt.Unf.SF', 'Total.Bsmt.SF',
       'Heating', 'Heating.QC', 'Central.Air', 'Electrical', 'X1st.Flr.SF',
       'X2nd.Flr.SF', 'Low.Qual.Fin.SF', 'Gr.Liv.Area', 'Bsmt.Full.Bath',
       'Bsmt.Half.Bath', 'Full.Bath', 'Half.Bath', 'Bedroom.AbvGr',
       'Kitchen.AbvGr', 'Kitchen.Qual', 'TotRms.AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace.Qu', 'Garage.Type', 'Garage.Yr.Blt',
    

In [31]:
# clustering에 사용할 features
features = ['Lot.Area', 'Total.Bsmt.SF', 'Gr.Liv.Area']

df2 = df[features]
df2 = df2.dropna(axis=0)

df2.head()

,Lot.Area,Total.Bsmt.SF,Gr.Liv.Area
0,31770,1080.0,1656
1,11622,882.0,896
2,14267,1329.0,1329
3,11160,2110.0,2110
4,13830,928.0,1629


In [32]:
X = df2.copy()


# clustering에 사용할 features
features = ['Lot.Area', 'Total.Bsmt.SF', 'Gr.Liv.Area']


# 데이터 표준화
X_scaled = X.loc[:, features]
X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)


# K-Means Clustering 수행
kmeans = KMeans(n_clusters=10, random_state=0)
X["Cluster"] = kmeans.fit_predict(X_scaled)


X.head(10)

,Lot.Area,Total.Bsmt.SF,Gr.Liv.Area,Cluster
0,31770,1080.0,1656,9
1,11622,882.0,896,8
2,14267,1329.0,1329,4
3,11160,2110.0,2110,6
4,13830,928.0,1629,0
5,9978,926.0,1604,0
6,4920,1338.0,1338,4
7,5005,1280.0,1280,4
8,5389,1595.0,1616,6
9,7500,994.0,1804,0


In [33]:
# 각 centroids까지의 거리를 features로 만들기

kmeans = KMeans(n_clusters=10, n_init=10, random_state=0)


# `fit_transform`을 통해 거리 구하기
X_cd = kmeans.fit_transform(X_scaled)


X_cd = pd.DataFrame(X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])])
X = X.join(X_cd)


X.head(10)

,Lot.Area,Total.Bsmt.SF,Gr.Liv.Area,Cluster,Centroid_0,Centroid_1,Centroid_2,Centroid_3,Centroid_4,Centroid_5,Centroid_6,Centroid_7,Centroid_8,Centroid_9
0,31770,1080.0,1656,9,2.975262,3.457403,16.782015,2.757248,2.976521,3.705021,2.867007,11.050455,3.280276,1.483931
1,11622,882.0,896,8,1.421765,4.217376,19.469640,2.584533,1.255307,1.403080,2.422163,13.028620,0.423670,4.453874
2,14267,1329.0,1329,4,1.474771,2.975105,18.993807,1.877738,0.667528,2.394689,1.139216,11.670978,1.413338,3.749911
3,11160,2110.0,2110,6,3.165268,1.437409,19.359027,2.487040,2.480980,4.405535,1.262548,9.705109,3.572953,4.456555
4,13830,928.0,1629,0,0.697022,2.974469,19.080044,1.164551,1.142543,1.815946,1.714976,11.965903,1.478490,3.746001
5,9978,926.0,1604,0,0.322745,3.072437,19.568693,1.204929,0.954585,1.593855,1.722530,12.181891,1.278197,4.229460
6,4920,1338.0,1338,4,1.413068,3.205027,20.174651,2.035664,0.559703,2.223282,1.382466,12.128918,1.315091,4.893201
7,5005,1280.0,1280,4,1.344155,3.346082,20.176582,2.086225,0.531848,2.075897,1.515084,12.280321,1.149059,4.905415
8,5389,1595.0,1616,6,1.863188,2.542392,20.075720,1.917225,1.057087,2.918381,0.859932,11.389729,2.070833,4.804422
9,7500,994.0,1804,0,0.643503,2.779475,19.859454,0.941477,1.150113,1.909826,1.640068,11.977463,1.676431,4.484549
